In [100]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
import plotly.offline as py
from collections import Counter

In [48]:
N = 10 # N is the number of relations of a character (enables us to delete low relationship)

### Load data & Convert to graph format 

In [105]:
df_grandet_surronding = pd.read_csv(r'res_surronding_PER/Balzac-Honore-de_Eugenie-Grandet.csv') # df surronding PER

In [106]:
# network as get dict of dict

In [107]:
def get_dict_of_dict(df_res, N):
    dict_res = {}
    for column_perso in df_res.columns:
        dict_res[column_perso] = dict(Counter(df_res[column_perso]).most_common(N))
    return dict_res

In [108]:
dict_grandet = get_dict_of_dict(df_grandet_surronding.dropna(), N)

In [109]:
# get weights with mention counter

In [110]:
df_grandet_mention = pd.read_csv(r'res_mention_PER/Balzac-Honore-de_Eugenie-Grandet.csv') # df mention PER (Counter)

In [111]:
dict_grandet_mention = dict(zip(list(df_grandet_mention.mention), list(df_grandet_mention['count'])))

### Visualisation 

In [112]:
GrandetGraph = nx.Graph()

In [113]:
# Add node for each character
for perso in dict_grandet_mention.keys():
    GrandetGraph.add_node(perso, size = dict_grandet_mention[perso])

In [114]:
GrandetGraph.nodes

NodeView(('<La fille-2687>', '<Eugénie-3301>', '<le père-2169>', '<monsieur Grandet-943>', '<Grandet au Tribunal de Première Instance-2317>', '<Eugénie-1205>', '<Eugénie et sa mère-1499>', '<Eugénie-1353>', '<un homme-191>', "<l'-2932>"))

In [115]:
# For each co-appearance between two characters, add an edge
for char in dict_grandet.keys():
    for co_char in dict_grandet[char].keys():
        
        # Only add edge if the count is positive
        if dict_grandet[char][co_char] > 0:
            GrandetGraph.add_edge(char, co_char, weight = dict_grandet[char][co_char])

In [116]:
GrandetGraph.edges()

EdgeView([('<Eugénie-3301>', '<Eugénie-3301>'), ('<Eugénie-3301>', '<Cornoiller-3306>'), ('<Eugénie-3301>', '<nos parents-3319>'), ('<Eugénie-3301>', "<l' héritière-3338>"), ('<Eugénie-3301>', "<la famille d' Aubrion-3337>"), ('<Eugénie-3301>', '<le facteur de poste-3308>'), ('<Eugénie-3301>', '<dont-3317>'), ('<Eugénie-3301>', '<monsieur des Grassins-3318>'), ('<Eugénie-3301>', '<beaucoup de monde-3336>'), ('<Eugénie-3301>', '<Nous-3346>'), ('<le père-2169>', '<le père-2169>'), ('<le père-2169>', '<lui-2132>'), ('<le père-2169>', '<madame de Mirbel-2184>'), ('<le père-2169>', '<elle-2181>'), ('<le père-2169>', '<- vousErreur-2179>'), ('<le père-2169>', '<ami-2183>'), ('<le père-2169>', '<tous deux-2182>'), ('<le père-2169>', '<qui-2185>'), ('<le père-2169>', '<nous-2178>'), ('<le père-2169>', '<femme aimante-2186>'), ('<monsieur Grandet-943>', '<monsieur Grandet-943>'), ('<monsieur Grandet-943>', '<son-987>'), ('<monsieur Grandet-943>', '<Charles-920>'), ('<monsieur Grandet-943>', '<m

In [117]:
# Get positions for the nodes in G
pos_ = nx.spring_layout(GrandetGraph)

In [123]:
def make_edge(x, y, text, width):
    
    '''Creates a scatter trace for the edge between x's and y's with given width

    Parameters
    ----------
    x    : a tuple of the endpoints' x-coordinates in the form, tuple([x0, x1, None])
    
    y    : a tuple of the endpoints' y-coordinates in the form, tuple([y0, y1, None])
    
    width: the width of the line

    Returns
    -------
    An edge trace that goes between x0 and x1 with specified width.
    '''
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue'),
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')

In [124]:
# For each edge, make an edge_trace, append to list
edge_trace = []
for edge in GrandetGraph.edges():
    
    if GrandetGraph.edges()[edge]['weight'] > 0:
        char_1 = edge[0]
        char_2 = edge[1]

        x0, y0 = pos_[char_1]
        x1, y1 = pos_[char_2]

        text   = char_1 + '--' + char_2 + ': ' + str(GrandetGraph.edges()[edge]['weight'])
        
        trace  = make_edge([x0, x1, None], [y0, y1, None], text,
                           0.3*GrandetGraph.edges()[edge]['weight']**1.75)

        edge_trace.append(trace)

In [125]:
GrandetGraph.nodes()[node]

{}

In [126]:
# Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 10,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
# For each node in midsummer, get the position and size and add to the node_trace
for node in GrandetGraph.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([5*GrandetGraph.nodes()[node]['size']])
    node_trace['text'] += tuple(['<b>' + node + '</b>'])

KeyError: 'size'

In [127]:
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)


fig = go.Figure(layout = layout)

for trace in edge_trace:
    fig.add_trace(trace)

fig.add_trace(node_trace)

fig.update_layout(showlegend = False)

fig.update_xaxes(showticklabels = False)

fig.update_yaxes(showticklabels = False)

fig.show()
py.plot(fig, filename='GrandetGraph.html')

'GrandetGraph.html'